In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Develop Model Context Protocol (MCP) Client and Server with Google Agent Development Kit (ADK)



<table align="left">
<td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/develop_mcp_with_gemini_and_adk.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fmcp%2Fdevelop_mcp_with_gemini_and_adk.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/mcp/develop_mcp_with_gemini_and_adk.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  
  
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/develop_mcp_with_gemini_and_adk.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/develop_mcp_with_gemini_and_adk.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/develop_mcp_with_gemini_and_adk.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/develop_mcp_with_gemini_and_adk.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/develop_mcp_with_gemini_and_adk.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/mcp/develop_mcp_with_gemini_and_adk.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Author(s) |
| --- |
| [Dave Wang](https://github.com/wadave) |

## Overview

Agent Development Kit (ADK) is a flexible and modular framework for developing and deploying AI agents. ADK can be used with popular LLMs and open-source generative AI tools and is designed with a focus on tight integration with the Google ecosystem and Gemini models. ADK makes it easy to get started with simple agents with Gemini models and Google AI tools while providing the control and structure needed for more complex agent architectures and orchestration. 

The Model Context Protocol (MCP) is an open standard that simplifies how AI assistants connect with external data, tools, and systems. It achieves this by standardizing the way applications provide contextual information to Large Language Models (LLMs), creating a vital interface for models to interact directly with various external services.

Developers building MCP-enabled applications have the flexibility to utilize existing third-party MCP servers or implement their own custom server solutions.

This notebook focuses on the latter, demonstrating how to build custom MCP servers using Gemini. Then we will show to how to use ADK with MCP clients to communicate with MCP servers.

#### MCP server code generation examples:
- Example 1: Building a BigQuery MCP Server
- Example 2: Building a MedlinePlus MCP Server
- Example 3: Building an NIH MCP Server
- Example 4: Building a Cocktail MCP Server

#### ADK and MCP client integration:
-  Use Google ADK single agent to test
-  Use Google ADK multi-agent to test

## Get started

### Install Google Gen AI SDK and other required packages


In [ ]:
%pip install --upgrade --quiet google-genai mcp geopy black google-cloud-bigquery google-adk

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import Libraries

In [ ]:
import sys
import os
import asyncio
import datetime
import json
import requests
import black
import re

from typing import Union, Dict, List, Optional
from google.genai.types import (
    GenerateContentConfig,
)
from google import genai
from google.genai import types
from typing import List, Dict, Any

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from IPython.display import display, Markdown

### Helper function


In [ ]:
def get_url_content(url):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)

        # Raise an exception if the request returned an error status code (like 404 or 500)
        response.raise_for_status()

        # Get the content of the response as text (HTML, in this case)
        # 'requests' automatically decodes the content based on HTTP headers
        file_content = response.text

        # Now you can work with the content
        print("Successfully fetched content")
        return file_content
        # Or, save it to a file:
        # with open("server_page.html", "w", encoding="utf-8") as f:
        #     f.write(file_content)
        # print("Content saved to server_page.html")

    except requests.exceptions.RequestException as e:
        # Handle potential errors during the request (e.g., network issues, DNS errors)
        print(f"Error fetching URL {url}: {e}")
    except requests.exceptions.HTTPError as e:
        # Handle HTTP error responses (e.g., 404 Not Found, 503 Service Unavailable)
        print(f"HTTP Error for {url}: {e}")


def format_python(raw_code, output_filename):

    try:
        # Format the code string using black
        # Use default FileMode which is generally recommended
        formatted_code = black.format_str(raw_code, mode=black.FileMode())

        # Save the formatted code to the specified file
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(formatted_code)

        print(f"Successfully formatted the code and saved it to '{output_filename}'")

    except black.InvalidInput as e:
        print(
            f"Error formatting code: The input string does not seem to be valid Python syntax."
        )
        print(f"Details: {e}")
    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


def extract_json_from_string(input_str: str) -> Optional[Union[Dict, List]]:
    """
    Extracts JSON data from a string, handling potential variations.

    This function attempts to find JSON data within a string. It specifically
    looks for JSON enclosed in Markdown-like code fences (```json ... ```).
    If such a block is found, it extracts and parses the content.
    If no code block is found, it attempts to parse the entire input string
    as JSON.

    Args:
        input_str: The string potentially containing JSON data. It might be
                   a plain JSON string or contain a Markdown code block
                   with JSON, possibly preceded by other text (like 'shame').

    Returns:
        The parsed JSON object (typically a dictionary or list) if valid
        JSON is found and successfully parsed.
        Returns None if no valid JSON is found, if parsing fails, or if the
        input is not a string.
    """
    if not isinstance(input_str, str):
        # Handle cases where input is not a string
        return None

    # Pattern to find JSON within ```json ... ``` blocks
    # - ````json`: Matches the start fence.
    # - `\s*`: Matches any leading whitespace after the fence marker.
    # - `(.*?)`: Captures the content (non-greedily) between the fences. This is group 1.
    # - `\s*`: Matches any trailing whitespace before the end fence.
    # - ` ``` `: Matches the end fence.
    # - `re.DOTALL`: Allows '.' to match newline characters.
    pattern = r"```json\s*(.*?)\s*```"
    match = re.search(pattern, input_str, re.DOTALL)

    json_string_to_parse = None

    if match:
        # If a markdown block is found, extract its content
        json_string_to_parse = match.group(
            1
        ).strip()  # Get captured group and remove surrounding whitespace
    else:
        # If no markdown block, assume the *entire* input might be JSON
        # We strip whitespace in case the string is just JSON with padding
        json_string_to_parse = input_str.strip()

    if not json_string_to_parse:
        # If after stripping, the potential JSON string is empty, return None
        return None

    try:
        # Attempt to parse the determined string (either from block or whole input)
        parsed_json = json.loads(json_string_to_parse)
        return parsed_json
    except json.JSONDecodeError:
        # Parsing failed, indicating the string wasn't valid JSON
        return None
    except Exception as e:
        # Catch other potential unexpected errors during parsing
        print(f"An unexpected error occurred during JSON parsing: {e}")
        return None


from pathlib import Path


def create_folder_if_not_exists(folder_path_str: str) -> bool:
    """
    Creates a folder (and any necessary parent folders) if it doesn't already exist.
    Uses print() for status and error messages.

    Args:
        folder_path_str (str): The path string for the folder to be created.
                               Can be relative or absolute.

    Returns:
        bool: True if the folder already exists or was successfully created,
              False if an error occurred during creation (e.g., permission denied).
    """
    try:
        # Convert the string path to a Path object
        folder_path = Path(folder_path_str)

        # Use mkdir() with options:
        # parents=True: Creates any necessary parent directories. Like 'mkdir -p'.
        # exist_ok=True: Doesn't raise an error if the directory already exists.
        folder_path.mkdir(parents=True, exist_ok=True)

        # Print confirmation (using resolve() to show the absolute path)
        print(f"Info: Successfully ensured folder exists: {folder_path.resolve()}")
        return True

    except PermissionError:
        print(
            f"Error: Permission denied: Could not create folder at '{folder_path_str}'."
        )
        return False
    except OSError as e:
        # Catch other OS-related errors (e.g., path is a file, invalid path format on Windows)
        print(f"Error: OS error creating folder '{folder_path_str}': {e}")
        return False
    except Exception as e:
        # Catch any other unexpected errors
        print(
            f"Error: An unexpected error occurred creating folder '{folder_path_str}': {e}"
        )
        return False

In [ ]:
create_folder_if_not_exists("server")

### Option 1 use a  Vertex AI project

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Option 2. Use a Google API Key 
Uncomment the following block to use Express Mode

In [ ]:
# API_KEY = "[your-api-key]"  # @param {type: "string", placeholder: "[your-api-key]", isTemplate: true}

# if not API_KEY or API_KEY == "[your-api-key]":
#     raise Exception("You must provide an API key to use Google AI in express mode.")

# client = genai.Client(api_key=API_KEY)

## Set up model id

In [ ]:
MODEL_ID = "gemini-2.5-pro-preview-03-25"

### Get system instruction context info

In [ ]:
# The URL you want to fetch
url = "https://modelcontextprotocol.io/quickstart/server"
reference_content = get_url_content(url)

### Set up system instruction

In [ ]:
from pydantic import BaseModel


class ResponseSchema(BaseModel):
    python_code: str
    description: str


system_instruction = f"""
  You are an MCP server export.
  Your mission is to write python code for MCP server.
  Here's the MCP server development guide and example
  {reference_content}
  
"""

#### Set function to generate MCP server code

In [ ]:
def generate_mcp_server(prompt):
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=GenerateContentConfig(
            system_instruction=system_instruction,
            response_mime_type="application/json",
            response_schema=ResponseSchema,
        ),
    )

    return response.text

## Generate MCP Server Code

### Example 1:  Build MCP Server for Google Cloud BigQuery

In [ ]:
prompt = """
  Please create an MCP server code for Google Cloud BigQuery. It has two tools. One is to list tables for all datasets, the other is to describe a table. Google Cloud project id and location will be provided in the query string. please use project id to access BigQuery client.
  Please output JSON output only.
  
"""

In [ ]:
response_text = generate_mcp_server(prompt)

In [ ]:
python_code = extract_json_from_string(response_text)["python_code"]
format_python(python_code, "server/bq.py")

### Example 2:  Build MCP server for Medlineplus website
Create an MCP server for 
https://medlineplus.gov/about/developers/webservices/ API service

In [ ]:
med_url = "https://medlineplus.gov/about/developers/webservices/"
med_prompt_base = """
  Please create an MCP server code for https://medlineplus.gov/about/developers/webservices/. It has one tool, get_medical_term. You provide a medical term, this tool will return explanation of the medial term.
  
  Here's the API details:
  
"""

prompt = [med_prompt_base, types.Part.from_uri(file_uri=med_url, mime_type="text/html")]
response_text = generate_mcp_server(prompt)
python_code = extract_json_from_string(response_text)["python_code"]

format_python(python_code, "server/med.py")

### Example 3: Build MCP Server for NIH

In [ ]:
nih_url = "https://clinicaltables.nlm.nih.gov/apidoc/icd10cm/v3/doc.html"
nih_prompt_base = """
  Please create an MCP server code for NIH. It has one tool, get_icd_10_code. You provide a name or code, it will return top 5 results. 
  
  Here's the API details:

"""
prompt = [nih_prompt_base, types.Part.from_uri(file_uri=nih_url, mime_type="text/html")]
response_text = generate_mcp_server(prompt)
python_code = extract_json_from_string(response_text)["python_code"]

format_python(python_code, "server/nih.py")

### Example 4: Build MCP Server for the Cocktail DB

In [ ]:
ct_url = "https://www.thecocktaildb.com/api.php"
ct_prompt_base = """
  Please create an MCP server code for the cocktail db. It has 5 tools:
  1. search cocktail by name
  2. list all cocktail by first letter
  3. search ingredient by name. 
  4. list random cocktails
  5. lookup full cocktail details by id
  
  Here's the API details:

"""
prompt = [ct_prompt_base, types.Part.from_uri(file_uri=ct_url, mime_type="text/html")]
response_text = generate_mcp_server(prompt)
python_code = extract_json_from_string(response_text)["python_code"]

format_python(python_code, "server/cocktail.py")

## Testing MCP Servers

### Testing with Google ADK
Note: It may not work in Colab (as of 4/16/2025) due to std io limitations. Please run it in Jupyter Notebook (VSCode, or Vertex AI workbench). 

In [ ]:
import os

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

In [ ]:
from google.adk.tools.mcp_tool.mcp_toolset import (
    MCPToolset,
    SseServerParams,
    StdioServerParameters,
)
from google.adk.agents.llm_agent import LlmAgent

import asyncio
from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
import contextlib
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService

load_dotenv()


async def get_tools_async(server_params):
    """Gets tools from MCP Server."""
    tools, exit_stack = await MCPToolset.from_server(connection_params=server_params)
    # MCP requires maintaining a connection to the local MCP Server.
    # Using exit_stack to clean up server connection before exit.
    return tools, exit_stack


async def get_agent_async(server_params):
    """Creates an ADK Agent with tools from MCP Server."""
    tools, exit_stack = await get_tools_async(server_params)
    root_agent = LlmAgent(
        model="gemini-2.5-pro-preview-03-25",
        name="ai_assistant",
        instruction="Use tools to get information to answer user questions",
        tools=tools,
    )
    return root_agent, exit_stack

In [ ]:
async def list_mcp_tools(server_params):
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:

            # Initialize the connection
            await session.initialize()

            # Get tools from MCP session and convert to Gemini Tool objects
            mcp_tools = await session.list_tools()

            return mcp_tools

### A Google ADK agent integrated with single MCP client

In [ ]:
async def run_adk_agent(server_params, question):
    session_service = InMemorySessionService()
    artifacts_service = InMemoryArtifactService()
    session = session_service.create_session(state={}, app_name="my_app", user_id="123")

    query = question
    print("[user]: ", query)
    content = types.Content(role="user", parts=[types.Part(text=query)])
    root_agent, exit_stack = await get_agent_async(server_params)
    runner = Runner(
        app_name="my_app",
        agent=root_agent,
        artifact_service=artifacts_service,
        session_service=session_service,
    )
    events_async = runner.run_async(
        session_id=session.id, user_id="123", new_message=content
    )

    async for event in events_async:
        # print(event)
        if event.content.role == "user" and event.content.parts[0].text:
            print("[user]:", event.content.parts[0].text)
        if event.content.parts[0].function_response:
            print("[-tool_response-]", event.content.parts[0].function_response)
        if event.content.role == "model" and event.content.parts[0].text:
            print("[agent]:", event.content.parts[0].text)

    await exit_stack.aclose()
    return events_async

In [ ]:
ct_server_params = StdioServerParameters(
    command="python",
    args=["./server/cocktail2.py"],
)

In [ ]:
events_async = await run_adk_agent(
    ct_server_params,
    "Please get cocktail margarita id and then full detail of cocktail margarita",
)

In [ ]:
bq_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/bq.py"],
)

In [ ]:
await run_adk_agent(
    bq_server_params,
    "Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'",
)

In [ ]:
med_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/med.py"],
)

In [ ]:
await run_adk_agent(med_server_params, "Please explain flu in detail.")

In [ ]:
nih_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/nih.py"],
)

In [ ]:
await run_adk_agent(nih_server_params, "Please tell me icd-10 code for pneumonia")

In [ ]:
ct_server_params = StdioServerParameters(
    command="python",
    args=["./server/cocktail2.py"],
)

In [ ]:
await run_adk_agent(
    ct_server_params,
    "Please get cocktail margarita id and then full detail of cocktail margarita",
)

### Test ADK multi-agent with MCP multi-client

In [ ]:
MODEL_ID = "gemini-2.0-flash"  
from pydantic import BaseModel
from typing import Dict


class AllServerConfigs(BaseModel):
    configs: Dict[str, StdioServerParameters]


async def run_multi_agent_with_mcp_clients(
    server_config_dict: AllServerConfigs, query: str
):
    session_service = InMemorySessionService()
    artifacts_service = InMemoryArtifactService()
    session = session_service.create_session(state={}, app_name="my_app", user_id="123")

    print("[user]: ", query)
    content = types.Content(role="user", parts=[types.Part(text=query)])

    all_tools = {}
    # Use a single ExitStack in the main task
    async with contextlib.AsyncExitStack() as stack:  # Master stack
        print("Setting up MCP connections sequentially...")
        for key, value in server_config_dict.items():
            server_params = value
            individual_exit_stack = (
                None  # Define outside try for broader scope if needed
            )
            try:
                # 1. AWAIT the call to run the function and get its results
                print(f"  Attempting connection for {server_params}...")
                tools, individual_exit_stack = await MCPToolset.from_server(
                    connection_params=server_params
                )

                # 2. Check if an exit stack was actually returned
                if individual_exit_stack is None:
                    print(
                        f"  Warning: No exit stack returned for {server_params}. Cannot manage cleanup."
                    )

                # 3. Enter the *returned* individual_exit_stack into the master stack
                #    This makes the master stack responsible for cleaning it up later.
                print(f"  Registering cleanup stack for {server_params}...")
                await stack.enter_async_context(individual_exit_stack)

                # 4. Add the tools
                print(f"  Connection established for {server_params}, got tools.")
                # Check if tools is None or empty if connection might partially fail
                if tools:
                    all_tools.update({key: tools})
                else:
                    print(
                        f"  Warning: Connection successful but no tools returned for {server_params}."
                    )

            except TypeError as te:
                # This error would now likely mean 'individual_exit_stack' is not a context manager
                print(f"TypeError during setup for {server_params}: {te}")

                # Decide whether to continue or raise
            except Exception as e:
                # Catch other errors during the MCPToolset.from_server call itself
                print(f"Error setting up connection for {server_params}: {e}")
                # Optionally re-raise if errors are critical: raise

        print(f"Finished setup. Collected {len(all_tools)} servers.")

        # --- Agent Creation and Run (remains the same) ---
        if not all_tools:
            print(
                "Warning: No tools were collected. Agent may not function as expected."
            )
            # Consider returning early or raising an error if tools are essential

        print(all_tools)
        booking_tools = all_tools["bnb"]
        booking_tools.extend(all_tools["weather"])

        ct_tools = all_tools["ct"]

        booking_agent = LlmAgent(
            model=MODEL_ID,
            name="booking_assistant",
            instruction="Use tools to get information to answer user questions",
            tools=booking_tools,
        )

        cocktail_agent = LlmAgent(
            model=MODEL_ID,
            name="cocktail_assistant",
            instruction="Use tools to get information to answer user questions",
            tools=ct_tools,
        )

        root_agent = LlmAgent(
            model=MODEL_ID,
            name="ai_assistant",
            instruction="""You have access to sub-agents named 'cocktail_assistant' and 'booking_assistant. 
            - If the user asks about cocktails, delegate the task
            to the 'cocktail_assistant' sub-agent. 
            - If the user asks about weather, room or house booking, delegate the task
            to the 'booking_assistant' sub-agent.
            - Carefully combine the information you find into a complete answer.
            - If you cannot find the specific information requested using your tools, let the user know.
            - Please format your response using Markdown to make it easy to read and understand.
            """,
            sub_agents=[cocktail_agent, booking_agent],
        )

        runner = Runner(
            app_name="my_app",
            agent=root_agent,
            artifact_service=artifacts_service,
            session_service=session_service,
        )

        print("Running agent...")
        events_async = runner.run_async(
            session_id=session.id, user_id="123", new_message=content
        )

        async for event in events_async:
            # Your event processing logic...
            if event.content.role == "user" and event.content.parts[0].text:
                print("[user]:", event.content.parts[0].text)
            if event.content.parts[0].function_response:
                print("[-tool_response-]", event.content.parts[0].function_response)
            if event.content.role == "model" and event.content.parts[0].text:
                print("[agent]:", event.content.parts[0].text)

        print("Agent run finished. Exiting context stack...")
        # Master stack cleanup happens automatically here

    print("Context stack closed, connections cleaned up.")
    return "Run completed"  # Or other appropriate return value

In [ ]:
# Create server parameters for stdio connection
weather_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your weather_server.py file
    args=["./server/weather_server.py"],
)

ct_server_params = StdioServerParameters(
    command="python",
    args=["./server/cocktail2.py"],
)
bnb_server_params = StdioServerParameters(
    command="npx", args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"]
)

server_config_dict = {
    "weather": weather_server_params,
    "bnb": bnb_server_params,
    "ct": ct_server_params,
}

In [ ]:
await run_multi_agent_with_mcp_clients(
    server_config_dict,
    "I want to book an airbnb apartment in LA, CA for 2 nights. 04/28 - 04/30, 2025, two adults, no kid",
)

### References:
https://google.github.io/adk-docs/
https://modelcontextprotocol.io/introduction  
https://github.com/modelcontextprotocol/python-sdk